In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import pymongo

In [2]:
##### FInd News Articles ###
url = "https://redplanetscience.com/"
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\steve\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


In [3]:
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, "html.parser")

In [5]:
soup_list = soup.find_all("div", class_="list_text")

In [6]:
headline = []
description = []

In [7]:
for i in soup_list:
    headlines = i.find("div", "content_title").text
    descriptions = i.find("div", "article_teaser_body").text
    headline.append(headlines)
    description.append(descriptions)

In [8]:
##### FInd Featured Photo ###
url = "https://spaceimages-mars.com/"

In [9]:
browser.visit(url)
time.sleep(0.025)

In [10]:
html = browser.html
soup = bs(html, "html.parser")

In [11]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [12]:
html = browser.html
soup = bs(html, "html.parser")

In [13]:
soup1 = soup.find("div", class_="floating_text_area")
photo_link = soup1.select_one("a",target_="_blank", class_="showimg fancybox-thumbs")["href"]

In [14]:
######## Find Mars Data #############
url = "https://galaxyfacts-mars.com/"
tables = pd.read_html(url)
tables

df = tables[0]
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df = df.set_index("Mars - Earth Comparison")
df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
######## Find  Hemisphere photos#############
url = "https://marshemispheres.com/"
browser.visit(url)
time.sleep(.025)
html = browser.html
soup = bs(html, "html.parser")
soupp = soup.find('div', class_='collapsible results')
items = soupp.find_all("div", class_="item")

In [16]:
scrape_link = []
picture_description = []
#image links
for i in items:
    desc_container = i.find("a", class_="itemLink product-item")['href']
    scrape_link.append(desc_container)
    picture_description.append(i.find("h3").text)

In [17]:
image_link = []
for i in scrape_link:
    scrapable_url = f'{url}{i}'
    browser.visit(scrapable_url)
    html = browser.html
    soup = bs(html, "html.parser")
    downloads = soup.find('div', class_='downloads')
    image_url = downloads.find("a")["href"]
    image_link.append(f'{url}{image_url}')
browser.quit()    

In [18]:
headline
description
photo_link
df
image_link

['https://marshemispheres.com/images/full.jpg',
 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg']

In [19]:
mars_dictionary = {
    "news": {
        "headline": headline,
        "description": description
    },
    "featured_photo" : photo_link,
    "data": df.to_dict(),
    "hemispheres": {
        "name": picture_description,
        "image_link":image_link
    }
}

In [20]:
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)
# db = client.Mars_Data
# Scrape_Results = db.Scrape_Results
# Scrape_Results.insert_one(mars_dictionary)

In [22]:
print(mars_dictionary["hemispheres"])

{'name': ['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced'], 'image_link': ['https://marshemispheres.com/images/full.jpg', 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg', 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg', 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg']}


In [37]:
# #zip picture description and image link together
# hem1 = zip(picture_description, image_link)
# #turn hem1 into a dictionary
# hemisphere_dict = dict(hem1)
# titles = []
# links = []
# for i in hemisphere_dict:
#     titles.append(i)
#     links.append(hemisphere_dict[i])
hemisphere_dict = []
for i in range(len(picture_description)):
    hemisphere_dict.append({"title":picture_description[i], "img_url":image_link[i]})
print(hemisphere_dict)



[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]
